In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import math
from math import trunc
from datetime import datetime, date, timedelta
import os
%matplotlib inline

In [ ]:
directory = '../data/aggregates/'

# CTIS data collected by CMU and Facebook:
US_ML = pd.concat([pd.read_csv(os.path.join(directory, filename)) for filename in os.listdir(directory) if os.path.isfile(os.path.join(directory, filename))])

regions1 = US_ML.state.unique()

In [ ]:
# Ground truth of serology data from CDC:
directory1 = '../data/'
US_official = pd.read_csv(os.path.join(directory1, "Nationwide_Commercial_Laboratory_Seroprevalence_Survey.csv"))

# NaNs in the database are represented by 777. We remove the empty rows.
US_official['Rate (%) [All Ages Cumulative Prevalence, Rounds 1-30 only]'] = US_official['Rate (%) [All Ages Cumulative Prevalence, Rounds 1-30 only]'].replace(777, np.nan)
US_official.drop(US_official[US_official['Round']>30].index,inplace=True)

US_official.drop(US_official[US_official['Estimated cumulative infections count'].isna()].index,inplace=True)
US_official['Estimated cumulative infections count'] = US_official['Estimated cumulative infections count'].apply(lambda x: int(x.replace(",","")))

regions2 = US_official.Site.unique()

In [ ]:
def format_date(date_string, start_end):
    if len(date_string) > 25:
        start_month_day_year, end_month_day_year =  list(map(lambda x: x.strip(), date_string.split(' - ')))
        start_month_day, start_year = start_month_day_year.split(', ')
        end_month_day, end_year = end_month_day_year.split(', ')
        start_date = datetime.strptime(start_month_day + ' ' + start_year,'%b %d %Y')
        end_date = datetime.strptime(end_month_day + ' ' + end_year,'%b %d %Y')
    else:
        month_day, year = date_string.split(', ')
        start_month_day, end_month_day = list(map(lambda x: x.strip(), month_day.split(' - ')))
        start_date = datetime.strptime(start_month_day + ' ' + year, '%b %d %Y')
        end_date = datetime.strptime(end_month_day + ' ' + year, '%b %d %Y')
    if start_end == 'start':
        return start_date.strftime('%Y-%m-%d')

    else:
        return end_date.strftime('%Y-%m-%d')

In [ ]:
# Save start and end dates of rounds:
US_official['start_date'] = US_official['Date Range of Specimen Collection'].map(lambda x: format_date(x, 'start'), na_action='ignore')
US_official['end_date'] = US_official['Date Range of Specimen Collection'].map(lambda x: format_date(x, 'end'), na_action='ignore')

In [ ]:
US_official['days between rounds'] = US_official['end_date'].map(lambda x: datetime.strptime(x, '%Y-%m-%d') if (type(x)== str) else timedelta(days=14)) - US_official['start_date'].map(lambda x: datetime.strptime(x, '%Y-%m-%d') if (type(x)== str) else timedelta(days=0))
US_official['days between rounds'] = US_official['days between rounds'].map(lambda x: x.days + 1)

In [ ]:
# Official data from Our World in Data:
US_cases = pd.read_csv(os.path.join(directory1, "United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv"))

#We adapt the date to the format we are using (yyyy-mm-dd):
US_cases['submission_date'] = US_cases['submission_date'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))

US_cases['new_case'] = US_cases['new_case'].apply(lambda x: int(x.replace(",","")))
US_cases['tot_death'] = US_cases['tot_death'].apply(lambda x: int(x.replace(",","")))

regions3 = US_cases.state.unique()

# Example to see data: Arkansas
data = US_cases[US_cases['state'] == 'AK'].sort_values(by='submission_date') #Take data chronologically
plt.plot( data['submission_date'], data['new_case']) #Plot of total cases in Arkansas

plt.show()

In [ ]:
# state_alphaCode sets the codes used to identify the states:
states = pd.read_csv(os.path.join(directory1, "State_codes.csv"))
states = states.drop(columns=['Numeric Code  '], axis=1)
states = states.rename(columns={' Name ' : 'State', ' Alpha Code ': 'Alpha Code'})
states.State = states.State.apply(lambda s: s.strip().capitalize())
states_unformatted = states.to_dict('records')
state_alphaCode = dict()

for unformatted_element in states_unformatted:
    state_alphaCode[unformatted_element['State']] = unformatted_element['Alpha Code'].strip()
    
# Add state codes in US_ML:
US_ML['state code']  = US_ML['state'].map(lambda x: x.capitalize()).replace(state_alphaCode)

In [ ]:
# Definite list of states to be used:
regions = list(US_ML['state code'].unique())
regions.sort()

In [ ]:
# COVID-19 cases estimations via wastewater SARS-CoV-2 concentration.
# WW_cases is the estimation for total infected population throughout time
WW_cases = pd.read_csv(os.path.join(directory1, "ww_estimate_infections.csv"))
WW_cases.drop(labels='id',axis=1,inplace=True) #We drop the states' id

WW_cases['state code']  = WW_cases['Country'].map(lambda x: x.capitalize()).replace(state_alphaCode) #adds the state code
#There are colonies, not only states and DC. remove the colonies:
for state_i in WW_cases['state code'].unique():
    if state_i not in state_alphaCode.values():
        WW_cases.drop(WW_cases[WW_cases['state code']==state_i].index,inplace=True)

In [ ]:
WW_new = pd.DataFrame(columns=['site','date','cases'])
dates_WW = WW_cases.columns[1:-1]
counter = 0
for state in WW_cases['state code'].unique():
    WW_new.loc[counter] = [state,dates_WW[0],WW_cases[WW_cases['state code']==state][dates_WW[0]].iloc[0]]
    counter += 1
    for date_i in range(1,len(dates_WW)):
        WW_new.loc[counter] = [state,dates_WW[date_i],WW_cases[WW_cases['state code']==state][dates_WW[date_i]].iloc[0]]
        counter += 1
WW_new.rename(columns={'cases': 'ww_cases'}, inplace=True) #rename "cases" so that it's not confused with other data
WW_cases = WW_new
#Now we have the same format as in the other dataframes

# Classes

In [ ]:
class AcumulatedIncidences:

    """dataframes to be used"""
    US_ML = US_ML
    US_official = US_official
    US_cases = US_cases
    WW_cases = WW_cases
    phi = (1+math.sqrt(5))/2 # for the purpose of making graph look nice
    regions = list(US_ML['state code'].unique()) #given all datasets have rows associated to these states, we will use these
    #Remember: regions=list(US_ML['state code'].unique())

    """
    define a start_date and an end_date to define the interval of study
    """


    def __init__(self,signals=['p_cli','p_rf','p_XGB','p_glm','new_case','ww_cases']):
        #signals are explanatory variables
        self.signals = signals
        self.incidenceVectors = dict()
        self.referenceVector = np.array([])
        self.incidenceDataFrame = None
        self.correlationFactors = dict()

    def getStartDate(self, d, region):
        #d = the date at the start of our interval
        # Finds the first available 'end_date' in the interval [d,Inf)
        start_date = US_official[(US_official['end_date'] >= d) & (US_official['Site'] == region)]['end_date'].iloc[0]
        return start_date


    def getEndDate(self, d, region):
        # Finds the last available 'end_date' in the interval (Inf,d]
        end_date = US_official[(US_official['end_date'] <= d) & (US_official['Site'] == region)]['end_date'].iloc[-1]
        return end_date


    def calculateDaysBetween(self, date1, date2):
        #returns the days between date1 and date2
        try:
            start_date = datetime.strptime(date1, '%Y-%m-%d')
            end_date = datetime.strptime(date2, '%Y-%m-%d')
            days = (end_date-start_date).days
        except TypeError:
            days = None
        return days

    def calculateVectorEntryForRegion(self, region, signal, start_date, end_date):
        #Gets number of infections for "region" between specified dates, using the specified model "signal". These values are the elements of the incidence vectors
        start_date = self.getStartDate(start_date,region)
        end_date = self.getEndDate(end_date,region)
        if signal in US_ML.columns:
            data = US_ML[(US_ML['date'] >= start_date) & (US_ML['date'] <= end_date) & (US_ML['state code'] == region)]
            cumulativeIntegral = data[signal].sum()
        elif signal in US_cases.columns:
            data = US_cases[(US_cases['submission_date'] >= start_date) & (US_cases['submission_date'] <= end_date) & (US_cases['state'] == region)]
            cumulativeIntegral = data[signal].sum()
        elif signal in WW_cases.columns:
            data = WW_cases[(WW_cases['date'] >= start_date) & (WW_cases['date'] <= end_date) & (WW_cases['site'] == region)]
            cumulativeIntegral = data[signal].sum()
        return cumulativeIntegral

    def MLEcoefs(self, reference_vector, incidence_matrix):
        # Estimates the coefficients of Linear Regression
        XtX = np.dot(incidence_matrix.T,incidence_matrix)
        Xty = np.dot(incidence_matrix.T,reference_vector)
        return np.linalg.solve(XtX,Xty)




class TemporalIncidences(AcumulatedIncidences):

    def __init__(self, region, interval_type='from0', signals=['p_cli','p_rf','p_XGB','p_glm','new_case','ww_cases'], rounds=None):
        self.region = region
        self.interval_type = interval_type
        self.incidenceMatrix = np.empty((0,len(signals)+1))
        if rounds is None:
            self.rounds = list(US_official[US_official['Site']==self.region]['Round'])
        else:
            self.rounds = rounds
        AcumulatedIncidences.__init__(self,signals)


    def calculateVectors(self):
        #Computes the incidence vectors: they're dicts where each pair of dates (start,end) has a value (that value is the % of infected people between those rounds and the selected region)
        for start_date, end_date in self.zipRounds():
            self.incidenceVectors['{0} {1}'.format(start_date, end_date)] = np.array([])
            for signal in self.signals:
                self.incidenceVectors['{0} {1}'.format(start_date, end_date)] = np.append(self.incidenceVectors['{0} {1}'.format(start_date, end_date)], self.calculateVectorEntryForRegion(self.region, signal, start_date, end_date))
        return self.incidenceVectors

    # single1 = Non-cumulative
    # from1 = Cumulative
    
    def zipRounds(self):
        #with this we can get the pairs of (start_date,end_date) with which we speccify the incidence vectors
        dates = list(US_official[(US_official['Site'] == self.region) & (US_official['Round'].map(lambda x: x in self.rounds))]['end_date'])

        if self.interval_type == 'single1':
            return zip(dates[:-1], dates[1:])
        elif self.interval_type == 'from1':
            return zip([dates[0] for i in range(len(dates)-1)], dates[1:])
        else:
            raise ValueError('unknown type of interval has been used', self.interval_type)


    def addReferenceVector(self):
        self.referenceVector = np.array([])
        for start_date, end_date in self.zipRounds():
            cumulativeIntegral = US_official[(US_official['end_date'] == end_date) & (US_official['Site'] == self.region)]['Rate (%) [All Ages Cumulative Prevalence, Rounds 1-30 only]'].iloc[0]
            self.referenceVector = np.append(self.referenceVector, cumulativeIntegral/100)
        return self.referenceVector

    def calculateIncidenceMatrix(self, normalize_everything=False):
        self.calculateVectors()
        self.incidenceMatrix = np.empty((0,len(self.signals)+1))
        for start_date, end_date in self.zipRounds():
            row = np.array([1]) # add a one if trying to add a coefficient
            row = np.append(row, self.incidenceVectors['{0} {1}'.format(start_date, end_date)])
            self.incidenceMatrix = np.vstack([self.incidenceMatrix, row])

        # Normilise tot_cases and ww_cases (or everything):
        to_normalize = self.signals if normalize_everything else ['new_case','ww_cases']
        for signal in to_normalize:
            if signal in self.signals:
                col_i = self.signals.index(signal)+1
                max_value = max(self.incidenceMatrix[:,col_i])
                self.incidenceMatrix[:,col_i] = self.incidenceMatrix[:,col_i]/max_value

        # Add days between data-points and real values:
        if self.interval_type == 'from1':
            # we have to remove the first column (constant)
            self.incidenceMatrix = np.delete(self.incidenceMatrix,0,1)
            a = np.array(US_official[(US_official['Site'] == self.region) & (US_official['Round'].map(lambda x: x in self.rounds))]['end_date'][1:].map(lambda x: datetime.strptime(x, '%Y-%m-%d')))
            first = datetime.strptime(US_official[(US_official['Site'] == self.region) & (US_official['Round'].map(lambda x: x in self.rounds))]['end_date'].iloc[0], '%Y-%m-%d')
            egunak = pd.array(a).map(lambda x: (x-first).days)
            self.incidenceMatrix = np.column_stack([ self.incidenceMatrix, egunak/max(egunak) ]) # weeks between rounds (float1 interval)
            self.incidenceMatrix = np.column_stack([ self.incidenceMatrix, np.array([US_official[(US_official['Site'] == self.region) & (US_official['Round'].map(lambda x: x in self.rounds))]['Rate (%) [All Ages Cumulative Prevalence, Rounds 1-30 only]'].iloc[0]/100 for i in range(len(a))]) ])

        elif self.interval_type == 'single1':
            a = np.array(US_official[(US_official['Site'] == self.region) & (US_official['Round'].map(lambda x: x in self.rounds))]['end_date'][1:].map(lambda x: datetime.strptime(x, '%Y-%m-%d')))
            b = np.array(US_official[(US_official['Site'] == self.region) & (US_official['Round'].map(lambda x: x in self.rounds))]['end_date'][:-1].map(lambda x: datetime.strptime(x, '%Y-%m-%d')))
            egunak = pd.array(a-b).map(lambda x: x.days)
            self.incidenceMatrix = np.column_stack([ self.incidenceMatrix, egunak/max(egunak) ]) # days between rounds (normilised)
            self.incidenceMatrix = np.column_stack([ self.incidenceMatrix, np.array(US_official[(US_official['Site'] == self.region) & (US_official['Round'].map(lambda x: x in self.rounds))]['Rate (%) [All Ages Cumulative Prevalence, Rounds 1-30 only]'][:-1])/100 ]) # zati 100 ehunekoa delako

        return self.incidenceMatrix


    def estimate(self, only_coefs=False, dates=False):
        first_end_date = self.getStartDate('2020-01-01',self.region)
        end_dates = [first_end_date] + [end for (start,end) in self.zipRounds()]

        ref_vec = self.addReferenceVector()
        inc_mat = self.calculateIncidenceMatrix()
        coeff = self.MLEcoefs(ref_vec,inc_mat)
        if only_coefs==True:
            if dates==True:
                return (coeff,end_dates)
            else:
                return coeff
        else:
            estim = np.dot(inc_mat,coeff)
            if dates==True:
                return (estim,end_dates)
            else:
                return estim




class AllRegionsAllRounds(AcumulatedIncidences):
    """
    Works with multiple states at once. Used for nationwide models. The states used can be specified.
    """

    def __init__(self, interval_type='single1', states=None, signals=['p_cli','p_rf','p_XGB','p_glm','new_case','ww_cases']):
        self.interval_type = interval_type
        self.incidenceMatrix = np.empty((0,len(signals)+1))
        AcumulatedIncidences.__init__(self,signals)
        self.regions = regions if states==None else states

    def zipRounds(self,region):
        #with this we can get the pairs of (start_round,end_round) with which we speccify the incidence vectors
        dates = list(US_official[US_official['Site'] == region]['end_date'])

        if self.interval_type == 'single1':
            return zip(dates[:-1], dates[1:])
        elif self.interval_type == 'from1':
            return zip([dates[0] for i in range(len(dates)-1)], dates[1:])
        else:
            raise ValueError('unknown type of interval has been used', self.interval_type)

    def calculateIncidenceMatrix(self, normalize_everything=False):
        self.incidenceMatrix = np.empty((0,len(self.signals)+3))
        
        # Normilise tot_cases and ww_cases (or everything):
        to_normalize = self.signals if normalize_everything else ['new_case','ww_cases']
        for region in self.regions:
            temp_mat = np.empty((0,len(self.signals)+1))
            for start_date, end_date in self.zipRounds(region):
                row = np.array([1]) # add a one if trying to add an intercept
                for signal in self.signals:
                    row = np.append(row, self.calculateVectorEntryForRegion(region, signal, start_date, end_date))
                temp_mat = np.vstack([temp_mat, row])
            for signal in to_normalize:
                if signal in self.signals:
                    col_i = self.signals.index(signal)+1
                    max_value = max(temp_mat[:,col_i])
                    temp_mat[:,col_i] = temp_mat[:,col_i]/max_value
                    
            # Add real values and days between data-points:
            a = np.array(US_official[US_official['Site']==region]['end_date'][1:])
            b = np.array(US_official[US_official['Site']==region]['end_date'][:-1])
            first = US_official[US_official['Site']==region]['end_date'].iloc[0]
            if self.interval_type == 'from1':
                egunak = np.array([self.calculateDaysBetween(first,xa) for xa in a])
                refs = np.array([US_official[US_official['Site']==region]['Rate (%) [All Ages Cumulative Prevalence, Rounds 1-30 only]'].iloc[0]/100 for i in range(len(a))])
            elif self.interval_type == 'single1':
                egunak = np.array([self.calculateDaysBetween(xb,xa) for (xa,xb) in zip(a,b)])
                refs = np.array([ xx/100 for xx in US_official[US_official['Site']==region]['Rate (%) [All Ages Cumulative Prevalence, Rounds 1-30 only]'][:-1] ])
            temp_mat = np.column_stack([ temp_mat, egunak/egunak.max() ])
            temp_mat = np.column_stack([ temp_mat, refs ])
            self.incidenceMatrix = np.vstack([self.incidenceMatrix, temp_mat])
            
        if self.interval_type=='from1':
            self.incidenceMatrix = np.delete(self.incidenceMatrix,0,1)


        return self.incidenceMatrix

    def addReferenceVector(self):
        self.referenceVector = np.array([])
        for region in self.regions:
            for start_date, end_date in self.zipRounds(region):
                cumulativeIntegral = US_official[(US_official['end_date'] == end_date) & (US_official['Site'] == region)]['Rate (%) [All Ages Cumulative Prevalence, Rounds 1-30 only]'].iloc[0]
                self.referenceVector = np.append(self.referenceVector, cumulativeIntegral/100)
        return self.referenceVector


    def estimate(self, only_coefs=False, inc_mat=None):
        ref_vec = self.addReferenceVector()
        if type(inc_mat)==type(None):
            inc_mat = self.calculateIncidenceMatrix()
        coeff = self.MLEcoefs(ref_vec,inc_mat)
        if only_coefs==True:
            return coeff
        else:
            estim = np.dot(inc_mat,coeff)
            return estim

# Neural Networks

In [ ]:
# STATEWIDE NN:

import torch
from torch.utils import data
from torch import nn
import copy

class nnExp(nn.Module):
    def __init__(self):
        super(nnExp, self).__init__()
    def forward(x):
    x = torch.exp(x)
        return x

class nnLn(nn.Module):
    def __init__(self):
        super(nnLn, self).__init__()
    def forward(x):
        zrk = torch.zeros(x.shape)
        x[x==zrk] = 1.0
        x = torch.log(torch.abs(x))
        return x


class NN_2layers:

    def __init__(self, state, interval, NN_type='relu', nodes_pl=6):
        self.state = state
        self.interval = interval
        self.nn_type = NN_type
        self.nodes_pl = nodes_pl

        instance = TemporalIncidences(self.state,interval_type=self.interval)
        self.helburua = instance.addReferenceVector()
        self.datuak = instance.calculateIncidenceMatrix(normalize_everything=True)
        if self.interval=='single1':
            self.datuak = self.datuak[:,1:]
        self.end_dates = instance.estimate(only_coefs=True,dates=True)[1]

    class Model(nn.Module):
        def __init__(self, NN_type='relu', nodes_pl=6):
            super().__init__()
            self.nn_type = NN_type
            self.nodes_pl = nodes_pl
            self.hidden1 = nn.Linear(8,nodes_pl)
            self.hidden2 = nn.Linear(nodes_pl,nodes_pl)
            self.output = nn.Linear(nodes_pl,1)
        def forward(self,x):
            half_nodes = self.nodes_pl//2
            third_nodes = self.nodes_pl//3
            linear_out = self.hidden1(x)
            if self.nn_type=='relu':
                x = nn.functional.relu(linear_out)
                linear_out = self.hidden2(x)
                x = nn.functional.relu(linear_out)

            elif self.nn_type=='mix':
                nodes_1 = linear_out[:,0:third_nodes]
                nodes_2 = linear_out[:,third_nodes:2*third_nodes]
                nodes_3 = linear_out[:,2*third_nodes:]
                x = torch.cat((nnExp.forward(nodes_1),nnLn.forward(nodes_2),nodes_3), dim=1)
                linear_out = self.hidden2(x)
                nodes_1 = linear_out[:,0:third_nodes]
                nodes_2 = linear_out[:,third_nodes:2*third_nodes]
                nodes_3 = linear_out[:,2*third_nodes:]
                x = torch.cat((nnExp.forward(nodes_1),nnLn.forward(nodes_2),nodes_3), dim=1)

            else:
                raise ValueError('unknown type of NN has been used, accepted types are "relu", "exp-ln", "ln-exp", "mix".', self.interval_type)
            x = self.output(x)
            return x

    def torcher(self,datuak,helburua):
        self.X = torch.tensor(datuak, dtype=torch.float32)
        y = torch.tensor(helburua, dtype=torch.float32)
        self.y = y.reshape((-1,1))
        return self.X, self.y

    def batcher(self,batch_size, data_arrays=None, is_train=True):
        if data_arrays==None:
            data_arrays = (self.X,self.y)
        dataset = data.TensorDataset(*data_arrays)
        return data.DataLoader(dataset, batch_size, shuffle=is_train)

    def initialise(self,learning_rate):
        self.model = self.Model(NN_type=self.nn_type,nodes_pl=self.nodes_pl)
        self.loss_function = nn.MSELoss()
        self.optimiser = torch.optim.SGD(self.model.parameters(), lr=learning_rate)

    def run_training(self, data_iter, num_epochs=5000, print_info=False):
        jarrai = True
        epoch = 0
        set_loss = float("inf")
        since_loss_update = 0
        while jarrai:
            train_loss = []
            self.model.train()
            for data,target in data_iter:
                self.optimiser.zero_grad()
                output = self.model(data)
                loss = self.loss_function(output,target)
                loss.backward()
                self.optimiser.step()
                train_loss.append(loss.item())
            if set_loss - np.mean(train_loss)>0.00001:
                set_loss = np.mean(train_loss)
                since_loss_update = 0
            else:
                since_loss_update += 1
            if since_loss_update == num_epochs:
                jarrai = False
            epoch += 1
            if print_info:
                if epoch%1000==0:
                    print("Epoch:",epoch, " Training Loss:", np.mean(train_loss))
        self.estimation = self.model(self.X).detach().numpy().T[0]
        return self.estimation
        
    def ploter(self, print_MARE=True):
        if print_MARE:
            print("MARE:",(np.abs(self.estimation-self.helburua)/self.helburua).mean())
        plt.plot(self.end_dates[1:], self.helburua, marker=".", linestyle="")
        plt.plot(self.end_dates[1:], self.estimation)
        plt.xticks(range(len(self.end_dates[1:])),rotation=90)
        plt.show()

    def reseter(self, reset_data=False):
        self.features = None
        self.labels = None
        self.model = None
        self.loss_function = None
        self.optimiser = None
        self.X = None
        self.y = None
        if reset_data:
            self.helburua = None
            self.datuak = None
            self.end_dates = None

In [ ]:
# Example of NN gradient descent for non-cumulative CA (2 layer relu with 6 neurons per layer):

state = 'CA'
typeNN = 'relu'
num_nodes = 6
interval = 'single1'

clcl = NN_2layers(state,interval,typeNN,num_nodes)

X_data,y_data = clcl.torcher(clcl.datuak,clcl.helburua)
trainloader = clcl.batcher(29, (X_data,y_data)

clcl.initialise(0.001)# <- learning rate
clcl.run_training(trainloader,5000, print_info=True)# <- num of epochs after which if there is no 0.00001 change it stops
print("·MARE:",(np.abs(clcl.estimation-clcl.helburua)/clcl.helburua).mean())

estim_list=np.array(clcl.estimation)
refvec_list=np.array(y_data).T[0]

y_bar = [refvec_list.mean() for i in range(len(refvec_list))]
SStot = np.linalg.norm(refvec_list - y_bar,2)**2
SSres = np.linalg.norm(refvec_list - estim_list,2)**2
print("·R^2:",1 - SSres/SStot)

In [ ]:
# USA-level NN:

class NN_2layersUSA:

    def __init__(self, states, interval, NN_type='relu', nodes_pl=6):
        self.states = states
        self.interval = interval
        self.nn_type = NN_type
        self.nodes_pl = nodes_pl

        instance = AllRegionsAllRounds(self.interval,self.states)
        self.helburua = instance.addReferenceVector()
        self.datuak = instance.calculateIncidenceMatrix(normalize_everything=True)
        if self.interval == 'single1':
                self.datuak = self.datuak[:,1:]
        
    class Model(nn.Module):
        def __init__(self, NN_type='relu', nodes_pl=6):
            super().__init__()
            self.nn_type = NN_type
            self.nodes_pl = nodes_pl
            self.hidden1 = nn.Linear(8,nodes_pl)
            self.hidden2 = nn.Linear(nodes_pl,nodes_pl)
            self.output = nn.Linear(nodes_pl,1)
        def forward(self,x):
            half_nodes = self.nodes_pl//2
            third_nodes = self.nodes_pl//3
            linear_out = self.hidden1(x)
            if self.nn_type=='relu':
                x = nn.functional.relu(linear_out)
                linear_out = self.hidden2(x)
                x = nn.functional.relu(linear_out)

            elif self.nn_type=='mix':
                nodes_1 = linear_out[:,0:third_nodes]
                nodes_2 = linear_out[:,third_nodes:2*third_nodes]
                nodes_3 = linear_out[:,2*third_nodes:]
                x = torch.cat((nnExp.forward(nodes_1),nnLn.forward(nodes_2),nodes_3), dim=1)
                linear_out = self.hidden2(x)
                nodes_1 = linear_out[:,0:third_nodes]
                nodes_2 = linear_out[:,third_nodes:2*third_nodes]
                nodes_3 = linear_out[:,2*third_nodes:]
                x = torch.cat((nnExp.forward(nodes_1),nnLn.forward(nodes_2),nodes_3), dim=1)

            else:
                raise ValueError('unknown type of NN has been used, accepted types are "relu", "mix".', self.interval_type)
            x = self.output(x)
            return x

    def torcher(self,datuak,helburua):
        self.X = torch.tensor(datuak, dtype=torch.float32)
        y = torch.tensor(helburua, dtype=torch.float32)
        self.y = y.reshape((-1,1))
        return self.X, self.y

    def batcher(self,batch_size, data_arrays=None, is_train=True):
        if data_arrays==None:
            data_arrays = (self.X,self.y)
        dataset = data.TensorDataset(*data_arrays)
        return data.DataLoader(dataset, batch_size, shuffle=is_train)
        
    def initialise(self,learning_rate):
        self.model = self.Model(NN_type=self.nn_type,nodes_pl=self.nodes_pl)
        self.loss_function = nn.MSELoss()
        self.optimiser = torch.optim.SGD(self.model.parameters(), lr=learning_rate)

    def run_training(self, data_iter, num_epochs=5000, print_info=False):
        jarrai = True
        epoch = 0
        set_loss = float("inf")
        since_loss_update = 0
        while jarrai:
            train_loss = []
            self.model.train()
            for data,target in data_iter:
                self.optimiser.zero_grad()
                output = self.model(data)
                loss = self.loss_function(output,target)
                loss.backward()
                self.optimiser.step()
                train_loss.append(loss.item())
            if set_loss - np.mean(train_loss)>0.00001:
                set_loss = np.mean(train_loss)
                since_loss_update = 0
            else:
                since_loss_update += 1
            if since_loss_update == num_epochs:
                jarrai = False
            epoch += 1
            if print_info:
                if epoch%1000==0:
                    print("Epoch:",epoch, " Training Loss:", np.mean(train_loss))
        self.estimation = self.model(self.X).detach().numpy().T[0]
        return self.estimation

    def reseter(self, reset_data=False):
        self.features = None
        self.labels = None
        self.model = None
        self.loss_function = None
        self.optimiser = None
        self.X = None
        self.y = None
        if reset_data:
            self.helburua = None
            self.datuak = None
            self.end_dates = None

In [ ]:
# Example of NN gradient descent for cumulative Top10 states (2 layer mixed with 6 neurons per layer):

top10 = ['CA','TX','FL','GA','NY','PA','IL','OH','MI','NC']
states = top10
typeNN = 'mix'
num_nodes = 6
interval = 'from1'

clcl = NN_2layersUSA(states,interval,typeNN,num_nodes)

X_data,y_data = clcl.torcher(clcl.datuak,clcl.helburua)
trainloader = clcl.batcher(clcl.datuak.shape[0], (X_data,y_data))

clcl.initialise(0.001)# <- learning rate
clcl.run_training(trainloader,5000, print_info=False)

print("·MARE:",(np.abs(clcl.estimation-clcl.helburua)/clcl.helburua).mean())

estim_list=np.array(clcl.estimation)
refvec_list=np.array(y_data).T[0]

y_bar = [refvec_list.mean() for i in range(len(refvec_list))]
SStot = np.linalg.norm(refvec_list - y_bar,2)**2
SSres = np.linalg.norm(refvec_list - estim_list,2)**2
print("·R^2:",1 - SSres/SStot)